# Lecture Overview


- Free cash flow to the firm (FCFF) approach
    - Calculating FCFFs
    - Discount rate is WACC
    - A simple example
    - Real-world implementation
    
    
- Free cash flow to equity holders (FCFE) approach
    - Calculating FCFEs
    - Discount rate is cost of equity
    - A simple example
    - Real-world implementation
        

In [1]:
# Import packages
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import statsmodels.api as sm

# Pretty print all cell's output and not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. The free cash flow to the firm (FCFF) approach

In this approach, we estimate the value of the firm's **assets** by discounting the firm's free cash flow to the firm (FCFF) at the firm's weighted average cost of capital (WACC): 

$$FirmValue_t = \frac{FCFF_{t+1}}{1+WACC} + \frac{FCFF_{t+2}}{(1+WACC)^2} + \frac{FCFF_{t+3}}{(1+WACC)^3} + ...$$        

To obtain the intrinsic value of of the firm's **equity**, we subtract the book value of debt (as an approximation of its market value of debt) from the $FirmValue$ estimate obtained as above. 

As we have seen in the DDM lecture, since we can not calculate infinite sums, in practice we often assume that, after some horizon $H$, the firm's FCFF grows at a constant rate $g$ for ever. That allow us to estimate the terminal value of the FCFFs at time $t+H$ as:

$$P_{t+H} = \frac{FCFF_{t+H}(1+g)}{(WACC - g)}$$

Alternatively, this terminal value can also be estimated using multiples (e.g. EV/EBITDA).

Once the terminal value is estimated, the valuation formula above becomes:

$$FirmValue_t = \frac{FCFF_{t+1}}{1+WACC} + \frac{FCFF_{t+2}}{(1+WACC)^2} + ... + \frac{FCFF_{t+H}}{(1+WACC)^{t+H}} + \frac{P_{t+H}}{(1+WACC)^{t+H}}$$        

The intermediate FCFFs (i.e. $FCFF_{t+1}$ up to $FCFF_{t+H}$) can be estimated either using pro-forma statements (as you learned in FIN 421) or (the approach taken here) by applying particular growth rates (which we estimate below) to the latest FCFF (i.e. $FCFF_{t}$).

Before we implement all of this, we show how we can calculate past FCFF for MSFT using its financial statements, and how to estimate its WACC.

### 1.1. Free cash flow to the firm (FCFF)

There are several definitions for FCFF, but the textbook one is:

$$FCFF = EBIT(1 - T_c) + Depreciation - Capital Expenditures - ChangeNWC$$

and $T_c$ is the firm's marginal tax rate, and $ChangeNWC$ is the change in the firm's net working capital.

In practice, we often don't have data on the firm's marginal tax rate and have to use its average tax rate instead (tax expense divided by taxable income). 

When we calculate net working capital, we commonly ignore cash and interest-bearing debt (i.e. NWC = (Current assets - Cash) - (Current Liabilities - Current debt) or NWC = Inventory + Accounts Receivable - Accounts Payable) but some analysts also use current assets minus current liabilities.

In [2]:
# load data from financial statements
financials = pd.read_pickle('./msft_all_statements.pkl')
#financials

In [3]:
# EBIT
ebit = financials.loc['Total Operating Profit/Loss', :]
ebit

2016    21292.0
2017    22632.0
2018    35058.0
2019    42959.0
2020    52959.0
Name: Total Operating Profit/Loss, dtype: float64

In [4]:
# Depreciation
depreciation = financials.loc['Depreciation, Amortization and Depletion, Non-Cash Adjustment', :]
depreciation

2016     6622.0
2017     8778.0
2018    10261.0
2019    11682.0
2020    12796.0
Name: Depreciation, Amortization and Depletion, Non-Cash Adjustment, dtype: float64

In [5]:
# CAPX
capx = -financials.loc['Purchase/Sale and Disposal of Property, Plant and Equipment, Net',:]
capx

2016     8343.0
2017     8129.0
2018    11632.0
2019    13925.0
2020    15441.0
Name: Purchase/Sale and Disposal of Property, Plant and Equipment, Net, dtype: float64

In [6]:
# Net working capital
c_assets = financials.loc['Total Current Assets',:]
cash = financials.loc['Cash, Cash Equivalents and Short Term Investments', :]
c_liab = financials.loc['Total Current Liabilities',:]
cdebt = financials.loc['Financial Liabilities, Current',:]
nwc = (c_assets - cash) - (c_liab - cdebt) 
nwc

change_nwc = nwc.diff()
change_nwc

2016   -20033.0
2017   -27536.0
2018   -18596.0
2019   -22171.0
2020   -23173.0
dtype: float64

2016       NaN
2017   -7503.0
2018    8940.0
2019   -3575.0
2020   -1002.0
dtype: float64

In [7]:
# Estimate tax rate
tax_expense = -financials.loc['Provision for Income Tax',:] 
tax_expense

taxable_income = financials.loc['Pretax Income',:]
taxable_income

tax_rates = tax_expense / taxable_income
tax_rates

# Take average of past tax rates to estimate the future ones
msft_T_c = tax_rates.mean()
msft_T_c

2016     2953.0
2017     1945.0
2018    19903.0
2019     4448.0
2020     8755.0
Name: Provision for Income Tax, dtype: float64

2016    19751.0
2017    23149.0
2018    36474.0
2019    43688.0
2020    53036.0
Name: Pretax Income, dtype: float64

2016    0.149511
2017    0.084021
2018    0.545676
2019    0.101813
2020    0.165077
dtype: float64

0.20921962078980286

In [8]:
# Calculate past FCFF
msft_FCFF = ebit * (1 - msft_T_c) + depreciation - capx - change_nwc
msft_FCFF

2016             NaN
2017    26048.941542
2018    17412.178534
2019    35303.134310
2020    40235.938103
dtype: float64

### 1.2. Weighted average cost of capital (WACC)

Denote by D and E the total debt and total equity of the firm respectively. Let $r_d$ denote the firm's cost of debt and $r_e$ its cost of equity. Then the firm's WACC is calculated as:

$$WACC = \frac{D}{D+E} r_d (1 - T_c) + \frac{E}{D+E} r_e$$

Above, D is usually measured as the book value of long-term debt (but we should use market value of debt if possible), and E is the market value of common equity. 

For the cost of debt $r_d$, we can use interest expense divided by long-term debt but a better approach is to find out if the firm has issued bonds and use the yield on those bonds as the firm's cost of debt. 

The cost of equity $r_e$ (i.e. the expected return on the stock), we usually use either CAPM or the Fama-French three factor model. 

In [9]:
# Book value of debt
lt_debt = financials.loc['Long Term Debt and Capital Lease Obligation','2020']
lt_debt_due = financials.loc['Current Portion of Long Term Debt and Capital Lease','2020']
D = lt_debt + lt_debt_due
D

70998.0

In [10]:
# Market value of equity (as of the end of the fiscal year)
msft_price = yf.download('MSFT','2020-06-01','2020-06-01', interval = '1mo')['Close']
msft_price

nr_shares = financials.loc['Diluted WASO','2020']
nr_shares

E = msft_price * nr_shares
E

[*********************100%***********************]  1 of 1 completed


Date
2020-06-01    203.509995
Name: Close, dtype: float64

7683.0

Date
2020-06-01    1.563567e+06
Name: Close, dtype: float64

In [11]:
# Market leverage
D/(D+E)

Date
2020-06-01    0.043435
Name: Close, dtype: float64

In [12]:
# Cost of debt
interest = -financials.loc['Net Interest Income/Expense',:]
interest_pct = interest / D
interest_pct

msft_r_d = interest_pct.mean()
msft_r_d

2016    0.017508
2017    0.031297
2018    0.038494
2019    0.037832
2020    0.036494
Name: Net Interest Income/Expense, dtype: float64

0.03232485422124566

In [13]:
# Cost of equity
msft_r_e = 0.0706 # from last class

In [14]:
# WACC
msft_wacc = msft_r_d * (1 - msft_T_c) * D / (D + E) + msft_r_e * E / (D + E)


msft_wacc

# Convert it from a Pandas series to a number (float)
msft_wacc = msft_wacc[0]
msft_wacc

Date
2020-06-01    0.068644
Name: Close, dtype: float64

0.06864375031478606

## 1.3. A simple example

Suppose in the latest fiscal year, MSFT had EBIT of \\$100 (all values are in billions), depreciation of $10, capital expenditures of \\$20, a change in net working capital (NWC) of \\$5, total debt of \\$70 and 2 billion shares outstanding . Assume the tax rate is 30\%. You believe the firm's FCFFs will grow at 10\% per year for the next 5 years, and at 4\% per year thereafter. The firm has a cost of debt of 3\% and a CAPM beta of 1.2. The current price of a share of MSFT is \\$200, and the current risk free rate is 0.001. Assume the market risk-premium is 6\%. What is the intrinsic value of one share of MSFT as of the end of the latest fiscal year?   

In [15]:
# Inputs
ebit = 100
depreciation = 10
capx = 20
changeNWC = 5
debt = 70
shares = 2
tax = 0.3

g1 = 0.10
g = 0.04

r_d = 0.03
beta = 1.2
p0 = 200
rf = 0.001
mkt_risk_prem = 0.06

In [16]:
# Calculate latest FCFF
FCFF0 = ebit * (1- tax) + depreciation - capx - changeNWC
FCFF0

55.0

In [17]:
# Calculate cost of equity using CAPM
r_e = rf + beta * mkt_risk_prem
r_e

0.073

In [18]:
# Calculate wacc
equity = p0 * shares

wacc = r_d * (1 - tax) * debt / (debt + equity) + r_e * equity / (debt + equity)
wacc

0.06525531914893618

In [19]:
# Calculate next 5 FCFFs
fcffs = pd.Series(np.nan, index = range(1,6))
fcffs

for t in range(1,6):
    fcffs[t] = FCFF0 * (1 + g1)**t
    
fcffs

1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
dtype: float64

1    60.50000
2    66.55000
3    73.20500
4    80.52550
5    88.57805
dtype: float64

In [20]:
# Calculate terminal value (as of 5 years from now)
p5 = fcffs[5] * (1 + g) / (wacc -g)
p5

3647.5948475147443

In [21]:
# Discount FCFFs and terminal value to today

    # Present value of next 5 FCFFs
pv_fcffs = 0
for t in range(1,6):
    pv_fcffs = pv_fcffs + fcffs[t] / ((1+wacc)**t)
pv_fcffs 

    # Present value of terminal value
pv_terminal = p5 / ((1 + wacc)**5)
pv_terminal

    # Intrinsic value of the firm
v_firm = pv_fcffs + pv_terminal
v_firm

303.1076489044273

2659.120611386511

2962.228260290938

In [22]:
# Calculate intrinsic value per share
v_equity = v_firm - debt
v_equity

v_per_share = v_equity / shares
v_per_share

2892.228260290938

1446.114130145469

## 1.4. Real-world implementation

We will use a three-stage growth model for FCFFs. We will assume the initial high-growth (g1 below) stage will last 5 years, followed by a more moderate growth stage (g2 below) that will last another 5 years, and a perpetual growth stage, at a rate slightly higher than the historical average growth rate in GDP (g = 0.04 below). We will assume that the intermediate growth rate (g2) will be a simple average of the initial growth rate (g1) and the perpetual growth rate (g). 

The first 3 sections below, estimate g1 in three different ways:
1. By taking an average of MSFTs past growth rates in FCFF
2. By taking an average of competitors' past growth rates in FCFFs
3. Using the industry's historical growth rate in EBIT

### 1.4.1. Estimate growth rate: using own firm data

In [23]:
# Using the FCFF we calculated above
msft_g1_own = msft_FCFF.pct_change().mean()
msft_g1_own

0.27855489635377034

In [24]:
# Using the FCFF provided by Morningstar (proccessed in the "Multiples" valuation lecture)
all_ratios = pd.read_pickle('./msft_ratios.pkl')
all_ratios

msft_fcff_mstar = all_ratios.loc['Free Cash Flow USD Mil',:]
msft_fcff_mstar

msft_g1_own = msft_fcff_mstar.pct_change().mean() #note that we are overwriting the previous msft_g1_own value
msft_g1_own 

,2011-06,2012-06,2013-06,2014-06,2015-06,2016-06,2017-06,2018-06,2019-06,2020-06
Revenue USD Mil,69943.00,73723.00,77849.00,86833.00,93580.00,85320.00,89950.00,110360.00,125843.00,143015.00
Gross Margin %,77.70,76.20,74.00,69.00,64.70,61.60,61.90,65.20,65.90,67.80
Operating Income USD Mil,27161.00,27956.00,26764.00,27886.00,28172.00,21292.00,22632.00,35058.00,42959.00,52959.00
Operating Margin %,38.80,37.90,34.40,32.10,30.10,25.00,25.20,31.80,34.10,37.00
Net Income USD Mil,23150.00,16978.00,21863.00,22074.00,12193.00,16798.00,21204.00,16571.00,39240.00,44281.00
Earnings Per Share USD,2.69,2.00,2.58,2.63,1.48,2.56,3.25,2.13,5.06,5.76
Dividends USD,0.61,0.76,0.89,1.07,1.21,1.39,1.53,1.65,1.80,1.99
Payout Ratio % *,22.60,26.40,44.30,38.20,49.00,101.50,66.10,109.50,39.10,32.30
Shares Mil,8593.00,8506.00,8470.00,8399.00,8254.00,8013.00,7832.00,7794.00,7753.00,7683.00
Book Value Per Share * USD,6.82,8.19,9.21,10.61,11.23,9.58,9.05,10.32,12.41,15.12


2011-06    24639.0
2012-06    29321.0
2013-06    24576.0
2014-06    26746.0
2015-06    23136.0
2016-06    24982.0
2017-06    31378.0
2018-06    32252.0
2019-06    38260.0
2020-06    45234.0
Name: Free Cash Flow USD Mil, dtype: float64

0.07930534415847572

### 1.4.2. Estimate growth rate: using competitor data

In [25]:
# Competitor FCFFs from Morningstar (go to Quote -> Key Ratios -> Full Key Ratios Data)
aapl_fcff = pd.Series([30077, 41454, 44590, 49900, 69778, 52276, 50803, 64121, 58896, 73365, 80219])
goog_fcff = pd.Series([11127,13346,11301,11417,16109,25824,23907,22832,30972,42843,42843])
amzn_fcff = pd.Series([2092,395,2031,1949,7331,9706,6479,17296,21653,25924,25924])

# Competitor FCFF growth rates
aapl_fcff_g1 = aapl_fcff.pct_change().mean()
aapl_fcff_g1

goog_fcff_g1 = goog_fcff.pct_change().mean()
goog_fcff_g1

amzn_fcff_g1 = amzn_fcff.pct_change().mean()
amzn_fcff_g1

# Average of competitor FCFF growth rates (excluding AMZN because it is such a large outlier)
msft_g1_comp = (aapl_fcff_g1 + goog_fcff_g1) / 2
msft_g1_comp

0.12121098169144369

0.16911047578615462

0.8161823773060606

0.14516072873879915

### 1.4.3. Estimate growth rate: using industry-level data

Please download the industry-level EBIT growth rates from D2L ("fundgrEB" under Lectures/Data) and save it in the same folder as these lecture notes (do not change the names of these files).

These files come from Aswath Damodaran at NYU. If you want to download these data yourself, go to 

http://pages.stern.nyu.edu/~adamodar/

Then hit "Data", then "Current Data", and download the "Fundamental Growth rate in EBIT" file ("US" one).

We will not import this Excel sheet into Python because we only need a single number from it so it's easier to just open the Excel file and look up the numbers we need. We use the fundamental growth rate in EBIT, estimated as the return on capital (ROC) times the retention ratio.

In [26]:
# Fundamental growth rate in EBIT = return on capital (ROC) times retention ratio 
msft_g1_ind = 0.075  # "Expected Growth in EBIT for Software (System & Application)

### 1.4.4. Estimating intermediate FCFFs

In [27]:
# Perpetual growth rate
msft_g = 0.04

# Intermediate growth rate (g2)
msft_g2_own = (msft_g1_own + msft_g) / 2
msft_g2_own

msft_g2_comp = (msft_g1_comp + msft_g) / 2
msft_g2_comp

msft_g2_ind = (msft_g1_ind + msft_g) / 2
msft_g2_ind

0.059652672079237856

0.09258036436939958

0.057499999999999996

In [28]:
# Extract latest FCFF
FCFF0 = msft_fcff_mstar[-1]
FCFF0


45234.0

In [29]:
# Calculate future 10 FCFFs, based on each of the 3 sets of growth rates (own, competitors, industry)

    # Create empty dataframe to store all our results   
fcffs = pd.DataFrame(np.nan, index = range(1,11), columns=['own','comp','ind'])
fcffs

    # Calculate FCFFs from t+1 to t+5
for t in range(1,6):
    fcffs.loc[t,'own'] = FCFF0 * (1+msft_g1_own)**t
    fcffs.loc[t,'comp'] = FCFF0 * (1+msft_g1_comp)**t
    fcffs.loc[t,'ind'] = FCFF0 * (1+msft_g1_ind)**t
fcffs    

    # Calculate FCFFs from t+6 to t+10
for t in range(6,11):
    fcffs.loc[t,'own'] = fcffs.loc[t-1,'own'] * (1+msft_g2_own)
    fcffs.loc[t,'comp'] = fcffs.loc[t-1,'comp'] * (1+msft_g2_comp)
    fcffs.loc[t,'ind'] = fcffs.loc[t-1,'ind'] * (1+msft_g2_ind)
fcffs    

,own,comp,ind
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN
10,NaN,NaN,NaN


,own,comp,ind
1,48821.297938,51800.200404,48626.550000
2,52693.087773,59319.555243,52273.541250
3,56871.931233,67930.425111,56194.056844
4,61382.179313,77791.255123,60408.611107
5,66250.114169,89083.490407,64939.256940
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN
10,NaN,NaN,NaN


,own,comp,ind
1,48821.297938,51800.200404,48626.550000
2,52693.087773,59319.555243,52273.541250
3,56871.931233,67930.425111,56194.056844
4,61382.179313,77791.255123,60408.611107
5,66250.114169,89083.490407,64939.256940
6,70202.110504,97330.872408,68673.264214
7,74389.853981,106341.800040,72621.976906
8,78827.407547,116186.962635,76797.740579
9,83529.673040,126943.593971,81213.610662
10,88512.441235,138696.078155,85883.393275


### 1.4.5. Estimating terminal value

In [30]:
# Create dataframe to store results
p10 = pd.DataFrame(np.nan, index = [10], columns = ['own','comp','ind'])
p10

# Calculate terminal value as of 10 years from now
for m in ['own','comp','ind']:
    p10.loc[10, m] = fcffs.loc[10,m] * (1 + msft_g) / (msft_wacc - msft_g)

p10

,own,comp,ind
10,NaN,NaN,NaN


,own,comp,ind
10,3.213718e+06,5.035790e+06,3.118262e+06


### 1.4.6. Intrinsic value of the firm

In [31]:
# Present value of next 10 FCFFs
    # Create dataframe to store results
pv_fcffs = pd.DataFrame(0, index = [10], columns = ['own','comp','ind'])
pv_fcffs

    # Calculate present values
for m in ['own','comp','ind']:
    for t in range(1,11):
        pv_fcffs.loc[10, m] = pv_fcffs.loc[10,m] + fcffs.loc[t,m] / ((1+msft_wacc)**t)

pv_fcffs

,own,comp,ind
10,0,0,0


,own,comp,ind
10,464777.332339,621372.191578,456026.788543


In [32]:
# Present value of terminal value
pv_terminal = p10 / ((1 + msft_wacc)**10)
pv_terminal

,own,comp,ind
10,1.654544e+06,2.592616e+06,1.605400e+06


In [33]:
# Intrinsic value of the firm
v_firm = pv_fcffs + pv_terminal
v_firm

,own,comp,ind
10,2.119321e+06,3.213988e+06,2.061426e+06


### 1.4.7. Intrinsic value of equity

In [34]:
# Load financial statements
financials = pd.read_pickle('./msft_all_statements.pkl')

# Extract data on debt and nr of shares outstanding
lt_debt = financials.loc['Long Term Debt and Capital Lease Obligation','2020']
lt_debt_due = financials.loc['Current Portion of Long Term Debt and Capital Lease','2020']
nr_shares = financials.loc['Diluted WASO','2020']

In [35]:
# Intrinsic value of equity
v_equity = v_firm - (lt_debt + lt_debt_due)
v_equity

,own,comp,ind
10,2.048323e+06,3.142990e+06,1.990428e+06


In [36]:
# Intrinsic value of one share of MSFT
v_per_share = v_equity / nr_shares
v_per_share

# Save results
v_per_share.to_pickle('./msft_valuation_DCF_FCFF')

,own,comp,ind
10,266.604603,409.083671,259.069165


# 2. The free cash flow to equity (FCFE) approach

In this approach, we estimate the value of the firm's **equity** directly by discounting the firm's free cash flow to equity (FCFE) at the firm's cost of equity ($r_e$): 

$$EquityValue_t = \frac{FCFE_{t+1}}{1+r_e} + \frac{FCFE_{t+2}}{(1+r_e)^2} + \frac{FCFE_{t+3}}{(1+r_e)^3} + ...$$        

To obtain the intrinsic value of one share, we divide to total value of equity by the number of shares outstanding. 

As we have seen in the DDM lecture, since we can not calculate infinite sums, in practice we often assume that, after some horizon $H$, the firm's FCFE grows at a constant rate $g$ for ever. That allow us to estimate the terminal value of the FCFEs at time $t+H$ as:

$$V_{t+H} = \frac{FCFE_{t+H}(1+g)}{(r_e - g)}$$

Alternatively, this terminal value can also be estimated using multiples (e.g. P/E or P/B).

Once the terminal value is estimated, the valuation formula above becomes:

$$EquityValue_t = \frac{FCFE_{t+1}}{1+r_e} + \frac{FCFE_{t+2}}{(1+r_e)^2} + ... + \frac{FCFE_{t+H}}{(1+r_e)^{t+H}} + \frac{V_{t+H}}{(1+r_e)^{t+H}}$$        

Before we implement this formula, we show how we can calculate past FCFE for MSFT using its financial statements (we have already estimated its cost of equity above)

### 2.1. Free cash flow to equity (FCFE)

There are several definitions for FCFE, but the textbook one is:

$$FCFE = FCFF - Interest (1 - T_c) + Debt Issuance$$

where Debt Issuance is positive if long-term debt increased from last year, and negative if it decreased. 

In [37]:
# Take a look at the ratios again
all_ratios

,2011-06,2012-06,2013-06,2014-06,2015-06,2016-06,2017-06,2018-06,2019-06,2020-06
Revenue USD Mil,69943.00,73723.00,77849.00,86833.00,93580.00,85320.00,89950.00,110360.00,125843.00,143015.00
Gross Margin %,77.70,76.20,74.00,69.00,64.70,61.60,61.90,65.20,65.90,67.80
Operating Income USD Mil,27161.00,27956.00,26764.00,27886.00,28172.00,21292.00,22632.00,35058.00,42959.00,52959.00
Operating Margin %,38.80,37.90,34.40,32.10,30.10,25.00,25.20,31.80,34.10,37.00
Net Income USD Mil,23150.00,16978.00,21863.00,22074.00,12193.00,16798.00,21204.00,16571.00,39240.00,44281.00
Earnings Per Share USD,2.69,2.00,2.58,2.63,1.48,2.56,3.25,2.13,5.06,5.76
Dividends USD,0.61,0.76,0.89,1.07,1.21,1.39,1.53,1.65,1.80,1.99
Payout Ratio % *,22.60,26.40,44.30,38.20,49.00,101.50,66.10,109.50,39.10,32.30
Shares Mil,8593.00,8506.00,8470.00,8399.00,8254.00,8013.00,7832.00,7794.00,7753.00,7683.00
Book Value Per Share * USD,6.82,8.19,9.21,10.61,11.23,9.58,9.05,10.32,12.41,15.12


In [38]:
# Interest percentage of revenue (from "all_ratios" dataframe)
interest_pct = all_ratios.loc['Net Int Inc & Other',:]
interest_pct

# Total revenue
revenue = all_ratios.loc['Revenue USD Mil',:]
revenue

# Interest in millions
msft_interest_expense = -revenue * interest_pct / 100
msft_interest_expense

2011-06     1.30
2012-06    -7.72
2013-06     0.37
2014-06    -0.08
2015-06   -10.33
2016-06    -1.81
2017-06     0.57
2018-06     1.28
2019-06     0.58
2020-06     0.05
Name: Net Int Inc & Other, dtype: float64

2011-06     69943.0
2012-06     73723.0
2013-06     77849.0
2014-06     86833.0
2015-06     93580.0
2016-06     85320.0
2017-06     89950.0
2018-06    110360.0
2019-06    125843.0
2020-06    143015.0
Name: Revenue USD Mil, dtype: float64

2011-06    -909.2590
2012-06    5691.4156
2013-06    -288.0413
2014-06      69.4664
2015-06    9666.8140
2016-06    1544.2920
2017-06    -512.7150
2018-06   -1412.6080
2019-06    -729.8894
2020-06     -71.5075
dtype: float64

In [39]:
# Calculate assets as revenues / asset turnover
a_turnover = all_ratios.loc['Asset Turnover',:]
a_turnover

total_assets = revenue / a_turnover
total_assets

2011-06    0.71
2012-06    0.64
2013-06    0.59
2014-06    0.55
2015-06    0.54
2016-06    0.46
2017-06    0.41
2018-06    0.44
2019-06    0.46
2020-06    0.49
Name: Asset Turnover, dtype: float64

2011-06     98511.267606
2012-06    115192.187500
2013-06    131947.457627
2014-06    157878.181818
2015-06    173296.296296
2016-06    185478.260870
2017-06    219390.243902
2018-06    250818.181818
2019-06    273571.739130
2020-06    291867.346939
dtype: float64

In [40]:
# Get long term debt 
debt_pct = all_ratios.loc['Long-Term Debt',:]
debt_pct

msft_debt = total_assets * debt_pct / 100
msft_debt

2011-06    10.97
2012-06     8.83
2013-06     8.85
2014-06    11.98
2015-06    15.78
2016-06    21.06
2017-06    31.55
2018-06    27.91
2019-06    23.26
2020-06    19.77
Name: Long-Term Debt, dtype: float64

2011-06    10806.686056
2012-06    10171.470156
2013-06    11677.350000
2014-06    18913.806182
2015-06    27346.155556
2016-06    39061.721739
2017-06    69217.621951
2018-06    70003.354545
2019-06    63632.786522
2020-06    57702.174490
dtype: float64

In [41]:
# Free cash flow to equity (FCFE)
msft_fcfe_mstar = msft_fcff_mstar - msft_interest_expense * (1 - msft_T_c) + msft_debt.diff()
msft_fcfe_mstar

2011-06             NaN
2012-06    24185.124314
2013-06    26309.657252
2014-06    33927.523516
2015-06    23924.022533
2016-06    35476.370370
2017-06    61939.345174
2018-06    34154.795284
2019-06    32466.614193
2020-06    39359.934696
dtype: float64

In [42]:
# Check how fast FCFEs have grown per year (on average)
msft_g1_own = msft_fcfe_mstar.pct_change().mean()
msft_g1_own

0.12820836872069855

## 2.2. A simple example

Suppose in the latest fiscal year, MSFT had EBIT of \\$100 (all values are in billions), depreciation of \\$10, capital expenditures of \\$20, a change in net working capital (NWC) of \\$5, total debt of \\$70 and 2 billion shares outstanding. In the latest fiscal year, the firm paid \\$10 in interest expense and in the year prior, it had total debt of \\$50. Assume the tax rate is 30\%. You believe the firm's FCFEs will grow at 10\% per year for the next 5 years, and at 4\% per year thereafter. The firm has a CAPM beta of 1.2 and the current risk free rate is 0.001. Assume the market risk-premium is 6\%. What is the intrinsic value of one share of MSFT as of the end of the latest fiscal year?   

In [43]:
# Inputs
ebit = 100
depreciation = 10
capx = 20
changeNWC = 5
debt = 70
debt_prior = 50
interest = 10

shares = 2
tax = 0.3

g1 = 0.10
g = 0.04

beta = 1.2
rf = 0.001
mkt_risk_prem = 0.06

In [44]:
# Calculate latest FCFE
FCFF0 = ebit*(1-tax) + depreciation - capx - changeNWC
FCFF0

55.0

In [45]:
# Calculate latest FCFE
FCFE0 = FCFF0 - interest*(1-tax) + (debt - debt_prior)
FCFE0

68.0

In [46]:
# Calculate cost of equity using CAPM
r_e = rf + beta * mkt_risk_prem
r_e

0.073

In [47]:
# Calculate next 5 dividends
fcfes = pd.Series(np.nan, index = range(1,6))
fcfes

for t in range(1,6):
    fcfes[t] = FCFE0 * (1 + g1)**t
    
fcfes

1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
dtype: float64

1     74.80000
2     82.28000
3     90.50800
4     99.55880
5    109.51468
dtype: float64

In [48]:
# Calculate terminal value (as of 5 years from now)
p5 = fcfes[5] * (1 + g) / (r_e -g)
p5

3451.371733333335

In [49]:
# Discount FCFEs and terminal value to today

    # Present value of next 5 FCFEs
pv_fcfes = 0
for t in range(1,6):
    pv_fcfes = pv_fcfes + fcfes[t] / ((1 + r_e)**t)
pv_fcfes 

    # Present value of terminal value
pv_terminal = p5 / ((1 + r_e)**5)
pv_terminal

    # Intrinsic value of the firm's equity
v_equity = pv_fcfes + pv_terminal
v_equity

366.54389830511985

2426.5716987440173

2793.1155970491372

In [50]:
# Calculate intrinsic value per share
v_per_share = v_equity / shares
v_per_share

1396.5577985245686

## 2.3. Real-world implementation

We will use a three-stage growth model for FCFEs. We will assume the initial high-growth (g1 below) stage will last 5 years, followed by a more moderate growth stage (g2 below) that will last another 5 years, and a perpetual growth stage, at a rate slightly higher than the historical average growth rate in GDP (g = 0.04 below). We will assume that the intermediate growth rate (g2) will be a simple average of the initial growth rate (g1) and the perpetual growth rate (g). 

For g1 we use the same values as we did for the FCFF approach, except for the g1 based on the firm's own historical growth rate: here we use the growth rate in past FCFE's (calculated in section 2.1.) instead of the growth rate in past FCFFs.

### 2.3.1. Estimating intermediate FCFFs

In [51]:
# Extract latest FCFE
FCFE0 = msft_fcfe_mstar[-1]
FCFE0

39359.93469602316

In [52]:
# Calculate future 10 FCFEs, based on each of the 3 sets of growth rates (own, competitors, industry)

    # Create empty dataframe to store all our results   
fcfes = pd.DataFrame(np.nan, index = range(1,11), columns=['own','comp','ind'])
fcfes

    # Calculate FCFEs from t+1 to t+5
for t in range(1,6):
    fcfes.loc[t,'own'] = FCFE0 * (1+msft_g1_own)**t
    fcfes.loc[t,'comp'] = FCFE0 * (1+msft_g1_comp)**t
    fcfes.loc[t,'ind'] = FCFE0 * (1+msft_g1_ind)**t
fcfes    

    # Calculate FCFEs from t+6 to t+10
msft_g2_own = (msft_g1_own + msft_g) / 2
msft_g2_own

for t in range(6,11):
    fcfes.loc[t,'own'] = fcfes.loc[t-1, 'own'] * (1 + msft_g2_own)
    fcfes.loc[t,'comp'] = fcfes.loc[t-1, 'comp'] * (1 + msft_g2_comp)
    fcfes.loc[t,'ind'] = fcfes.loc[t-1, 'ind'] * (1 + msft_g2_ind)
    
fcfes

,own,comp,ind
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN
10,NaN,NaN,NaN


,own,comp,ind
1,44406.207716,45073.451500,42311.929798
2,50099.455169,51616.346566,45485.324533
3,56522.624590,59109.013048,48896.723873
4,63769.298084,67689.320458,52563.978164
5,71945.055766,77515.151543,56506.276526
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN
10,NaN,NaN,NaN


0.08410418436034928

,own,comp,ind
1,44406.207716,45073.451500,42311.929798
2,50099.455169,51616.346566,45485.324533
3,56522.624590,59109.013048,48896.723873
4,63769.298084,67689.320458,52563.978164
5,71945.055766,77515.151543,56506.276526
6,77995.936000,84691.532517,59755.387426
7,84555.720581,92532.305456,63191.322203
8,91667.210493,101098.980012,66824.823230
9,99376.806464,110458.760418,70667.250565
10,107734.811716,120685.072706,74730.617473


### 2.3.2. Estimating terminal value

In [53]:
# Create dataframe to store results
p10 = pd.DataFrame(np.nan, index = [10], columns = ['own','comp','ind'])
p10

# Calculate terminal value as of 10 years from now
for m in ['own','comp','ind']:
    p10.loc[10, m] = fcfes.loc[10,m] * (1 + msft_g) / (msft_r_e - msft_g)

p10

,own,comp,ind
10,NaN,NaN,NaN


,own,comp,ind
10,3.661575e+06,4.101715e+06,2.539864e+06


### 2.3.3. Intrinsic value of the firm's equity

In [54]:
# Present value of next 10 FCFEs
    # Create dataframe to store results
pv_fcfes = pd.DataFrame(0, index = [10], columns = ['own','comp','ind'])
pv_fcfes

    # Calculate present values
for m in ['own','comp','ind']:
    for t in range(1,11):
        pv_fcfes.loc[10, m] = pv_fcfes.loc[10,m] + fcfes.loc[t,m] / ((1+msft_r_e)**t)

pv_fcfes

,own,comp,ind
10,0,0,0


,own,comp,ind
10,496545.820644,534947.677753,392861.92086


In [55]:
# Present value of terminal value
pv_terminal = p10 / ((1 + msft_r_e)**10)
pv_terminal

,own,comp,ind
10,1.850954e+06,2.073448e+06,1.283920e+06


In [56]:
# Intrinsic value of the firm
v_equity = pv_fcfes + pv_terminal
v_equity

,own,comp,ind
10,2.347500e+06,2.608395e+06,1.676782e+06


In [57]:
# Intrinsic value of one share of MSFT
v_per_share = v_equity / nr_shares
v_per_share

# Save results
v_per_share.to_pickle('./msft_valuation_DCF_FCFE')

,own,comp,ind
10,305.54466,339.502193,218.245777
